In [1]:
# 导入lib
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader,random_split
from torchvision import transforms
from tqdm import tqdm
import os
import csv
import torchvision.utils as vutils
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [6]:
from torchvision import datasets
import os

# 创建 ImageFolder 实例（无需 transform，仅提取类别映射）
dataset = datasets.ImageFolder(os.path.join('datasets', 'train'))

# 打印类别到索引的映射（中文文件夹名 -> 整数标签）
class_to_idx = dataset.class_to_idx
print("类别名称 → 类别索引 映射：")
for name, idx in class_to_idx.items():
    print(f"{name} → {idx}")


类别名称 → 类别索引 映射：
刀片 → 0
剩菜剩饭 → 1
厨房废品 → 2
大骨头 → 3
玻璃瓶 → 4
瓜果皮壳 → 5
肉类 → 6
茶叶渣 → 7
菜叶菜帮 → 8
鱼骨鱼刺 → 9


In [2]:
def load_data(batch_size=128):
    """导入数据集"""
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3./4., 4./3.)),
        transforms.RandomHorizontalFlip(p=0.5),
        AutoAugment(policy=AutoAugmentPolicy.IMAGENET), 
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])

    transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])
    train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
        os.path.join('datasets', folder),
        transform=transform_train) for folder in ['train', 'train_valid']]
    
    valid_ds, test_ds = [torchvision.datasets.ImageFolder(
        os.path.join('datasets', folder),
        transform=transform_test) for folder in ['valid', 'test']]
    num_workers =8
    train_iter, train_valid_iter = [torch.utils.data.DataLoader(
        dataset, batch_size, shuffle=True, drop_last=True)
        for dataset in (train_ds, train_valid_ds)]

    valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                            drop_last=True,num_workers=num_workers,
                                            pin_memory=True,  prefetch_factor=8,
                                            persistent_workers=True)

    test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                            drop_last=False,num_workers=num_workers,
                                            pin_memory=True,prefetch_factor=8,
                                            persistent_workers=True)
    return (train_iter, train_valid_iter, valid_iter,test_iter)

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
train_iter, train_valid_iter, valid_iter,test_iter = load_data(batch_size=64)
# for i,(X,y) in enumerate(train_iter): # 这里费时间,如何找到合适的num_workers
#     # X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
#     X, y = X[:8], y[:8]
#     print("Batch shape:", X.shape, "Labels shape:", y)
#     grid_img = vutils.make_grid(X, nrow=4, normalize=True)  # nrow=4 表示每行 4 张
#     plt.figure(figsize=(8, 4))  # 调整图像大小
#     plt.imshow(grid_img.permute(1, 2, 0))  # 调整通道顺序 (C, H, W) → (H, W, C)
#     plt.axis("off")
#     plt.show()
#     break


In [3]:
# 网络
finetune_net = torchvision.models.resnet50(weights=True)
finetune_net.fc = nn.Linear(finetune_net.fc.in_features, 10)
nn.init.xavier_uniform_(finetune_net.fc.weight);
finetune_net;

d:\Miniconda3\envs\yolov8_2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
def train_fromKK(net, train_iter, test_iter, num_epochs, lr, device,param_group=True):
    with open('training_metrics.txt', 'w') as f:
        f.write("Epoch,Train_Loss,Train_Acc,Test_Acc\n") 
    print('training on', device)
    net.to(device)
    loss = nn.CrossEntropyLoss()
    best_weights = 0
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        optimizer = torch.optim.AdamW([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': lr * 10}],
                                lr=lr, weight_decay=0.001)
    else:
        optimizer = torch.optim.AdamW(net.parameters(), lr=lr,
                                  weight_decay=0.001)
    for epoch in range(num_epochs):
        net.train()
        train_loss_sum, train_acc_sum,num_samples = 0,0,0
        with tqdm(train_iter, desc=f"Epoch {epoch+1}/{num_epochs}") as pbar:  
            for X, y in pbar:
                optimizer.zero_grad()
                X,y = X.to(device),y.to(device)
                y_hat = net(X)
                l = loss(y_hat, y)
                l.backward()
                optimizer.step()
                train_loss_sum += l.item() * X.shape[0]
                train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
                num_samples += X.shape[0]
                pbar.set_postfix(loss=l.item(), acc=train_acc_sum / num_samples)
        train_loss = train_loss_sum / num_samples
        train_acc = train_acc_sum / num_samples

        if test_iter is not None:
            net.eval()  # 评估模式
            test_acc_sum, test_samples = 0, 0
            with torch.no_grad():
                for X, y in test_iter: # 这里也很费时间，连续运行效率极高
                    X, y = X.to(device), y.to(device)
                    y_hat = net(X)
                    test_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
                    test_samples += X.shape[0]
            test_acc = test_acc_sum / test_samples
            if (test_acc>best_weights):
                best_weights = test_acc
                torch.save(net.state_dict(), 'test_best.pth')
            print(f"______ | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
            with open('training_metrics.txt', 'a') as f:
                f.write(f"{epoch+1},{train_loss:.4f},{train_acc:.4f},{test_acc:.4f}\n")
        else:
            torch.save(net.state_dict(), 'test_best.pth')
            print(f"______ | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

In [ ]:
# 训练
lr,num_epochs =5e-5,10
net = finetune_net
train_fromKK(net,train_iter,valid_iter,num_epochs,lr,device)

resNe18
test_best_5epoch.pt
______ | Train Loss: 0.5319 | Train Acc: 0.8173 | Test Acc: 0.8885
验证集精度：0.8885

resNe50
______ | Train Loss: 0.1763 | Train Acc: 0.9406 | Test Acc: 0.8997
验证集精度：0.9064


In [5]:
# 确定超参数后的训练
"""
超参数群
lr,num_epochs =5e-5,10
"""
lr,num_epochs =5e-5,5
net = finetune_net
train_fromKK(net,train_valid_iter,None,num_epochs,lr,device)



training on cuda


Epoch 1/5: 100%|██████████| 257/257 [02:55<00:00,  1.47it/s, acc=0.74, loss=0.635] 


______ | Train Loss: 0.7765 | Train Acc: 0.7402


Epoch 2/5: 100%|██████████| 257/257 [03:04<00:00,  1.40it/s, acc=0.811, loss=0.493]


______ | Train Loss: 0.5613 | Train Acc: 0.8110


...

______ | Train Loss: 0.4968 | Train Acc: 0.8271


Epoch 4/5: 100%|██████████| 257/257 [03:03<00:00,  1.40it/s, acc=0.847, loss=0.459]


______ | Train Loss: 0.4561 | Train Acc: 0.8465


Epoch 5/5: 100%|██████████| 257/257 [03:02<00:00,  1.41it/s, acc=0.851, loss=0.275]

______ | Train Loss: 0.4359 | Train Acc: 0.8510
